In [19]:
#from google.colab import drive

#drive.mount('/content/drive')

Mounted at /content/drive


In [20]:
#!cp /content/drive/MyDrive/archive.zip .

In [21]:
#!unzip archive.zip

Archive:  archive.zip
  inflating: completeSpamAssassin.csv  
  inflating: enronSpamSubset.csv     
  inflating: lingSpam.csv            


In [31]:
import pandas as pd

path="/content"

files=["completeSpamAssassin.csv", "enronSpamSubset.csv", "lingSpam.csv"]

enron_spam_subset_df=pd.read_csv("/content/"+files[0])
ling_spam_df=pd.read_csv("/content/"+files[1])
complete_spam_assassin_df=pd.read_csv("/content/"+files[2])

In [38]:
import pandas as pd

# All the "Unnamed columns are indexes, so we can drop them"
enron_spam_subset_df.drop(['Unnamed: 0'], axis=1, inplace=True)
ling_spam_df.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1, inplace=True)
complete_spam_assassin_df.drop(['Unnamed: 0'], axis=1, inplace=True)

# Combine the datasets into a single DataFrame
combined_df = pd.concat([ling_spam_df, enron_spam_subset_df, complete_spam_assassin_df], ignore_index=True)

combined_df.info(), combined_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18651 entries, 0 to 18650
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Body    18650 non-null  object
 1   Label   18651 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 291.5+ KB


(None,
                                                 Body  Label
 0  Subject: stock promo mover : cwtd\n * * * urge...      1
 1  Subject: are you listed in major search engine...      1
 2  Subject: important information thu , 30 jun 20...      1
 3  Subject: = ? utf - 8 ? q ? bask your life with...      1
 4  Subject: " bidstogo " is places to go , things...      1)

In [67]:
print(combined_df.isnull().sum())
combined_df.dropna(inplace=True)

In [68]:
combined_df

,Body,Label
0,Subject: stock promo mover : cwtd\n * * * urge...,1
1,Subject: are you listed in major search engine...,1
2,"Subject: important information thu , 30 jun 20...",1
3,Subject: = ? utf - 8 ? q ? bask your life with...,1
4,"Subject: "" bidstogo "" is places to go , things...",1
...,...,...
18646,Subject: computationally - intensive methods i...,0
18647,Subject: books : a survey of american linguist...,0
18648,Subject: wecol ' 98 - - western conference on ...,0
18649,Subject: euralex ' 98 - revised programme\n \n...,0


In [69]:
import plotly.express as px

label_counts = combined_df['Label'].value_counts().reset_index()
label_counts.columns = ['Label', 'Count']

label_counts['Color'] = label_counts['Label'].map({0: 'blue', 1: 'red'})

fig = px.bar(label_counts,
             x='Label',
             y='Count',
             title="Count of Labels",
             color='Color',
             color_discrete_map={'blue': 'blue', 'red': 'red'})

fig.show()

In [70]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(combined_df['Body'], combined_df['Label'], test_size=0.2, random_state=42)

In [71]:
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

model = make_pipeline(TfidfVectorizer(), MultinomialNB())
model.fit(X_train, y_train)

Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()),
                ('multinomialnb', MultinomialNB())])

In [72]:
from sklearn.metrics import classification_report, accuracy_score


predictions = model.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
report = classification_report(y_test, predictions)

In [73]:
accuracy

0.9010723860589812

In [77]:
accuracy_score(y_train, model.predict(X_train))

0.9291554959785523